In [1]:
%matplotlib inline
import json
import plotly.express as px
import plotly.graph_objects as go
import os
import pandas as pd
from statistics import mean, median, stdev, variance
import qgrid
from enum import Enum, IntEnum

In [2]:
# folder paths
results_folder = 'data/benchmark/'
cnf_path = 'data/cnf-instances/'
wcnf_path = 'data/wcnf-instances/'



# problem names
# p = set(f_name for f_name in gurobi_ext4_files)
f_paths = None

# extension
if os.path.exists(results_folder):
    f_paths = [f"{results_folder}{f}" for f in os.listdir(results_folder) if f.endswith('.json')]
   
cnf_files = [f.replace('.cnf', '') for f in os.listdir(cnf_path) if f.endswith('.cnf')]
wcnf_files = [f.replace('.wcnf', '') for f in os.listdir(wcnf_path) if f.endswith('.wcnf')]

cnf_problems = [f for f in f_paths if any(cnf_file in f for cnf_file in cnf_files)]
wcnf_problems = [f for f in f_paths if any(wcnf_file in f for wcnf_file in wcnf_files)]
#print(f_paths)

In [3]:
class ClauseCounting(IntEnum):
    VALIDATED = 1
    WEIGHTS = 2
    WEIGHTED_UNASSIGNED = 3

class ClauseSorting(IntEnum):
    IGNORE = 0
    WEIGHTS = 1
    UNASSIGNED = 2
    WEIGHTED_UNASSIGNED = 3
    LITERAL_ORDERING = 4

class BestLiteral(IntEnum):
    COUNT_PURE_ONLY = 1
    COUNT_POLARITY = 2

class UnitLiteral(IntEnum):
    IGNORE = 0
    RANDOM = 1
    SINGLE_POLARITY = 2
    POLARITY = 3
    IMMEDIATE = 4

class SatModel(IntEnum):
    RANDOM = 1
    BEST_CLAUSES_VALIDATED = 2
    BEST_CLAUSE_WEIGHTS_COVERAGE = 3
    BEST_WEIGHTED_UNASSIGNED_CLAUSE_COVERAGE = 4
    ALL = 5



data = {'p':  [],
        'solver': [],
        'type':[],
        'ext' : [],
        'clauses' : [],
        'steps': [],
        'total time [s]': [],
        '% hs [s]' : [],
        '% sat [s]' : [],
        '% grow [s]':[],
        's_hs':[],
        's_grow':[],
        'avg_s_hs':[],
        'avg_s_grow':[],
        'OMUS':[],
        "count_clauses": [],
        "best_unit_literal": [],
        "best_counter_literal": [],
        "sorting": [],
        "extension": [],
        "cutoff_main": [],
        "cutoff": [],
        "h_inc": [],
        "max_restart": [],
        "s_inc": [],
        "pb_restarts": [],
        "sp": [],
        "creation date": []
        }

for f_path in f_paths:
    p_name = f_path.replace('data/benchmark/','')
    data["creation date"].append(os.path.getctime(f_path))

    if f_path in cnf_problems:
        data['type'].append('cnf')
        data['p'].append([cnf_file for cnf_file in cnf_files if cnf_file in f_path][0])
    else:
        data['type'].append('wcnf')
        data['p'].append([wcnf_file for wcnf_file in wcnf_files if wcnf_file in f_path][0])

    with open(f_path) as f:
        parsed_json = json.load(f)
    if 'clauses' not in parsed_json:
        continue

    data['solver'].append('Gurobi')
    data['ext'].append(parsed_json['parameters']['extension'])
    data['clauses'].append(parsed_json['clauses'])
    data['steps'].append(parsed_json['steps'])
    tot_time = sum(parsed_json['t_hitting_set']) +sum(parsed_json['t_sat_check'])  + sum(parsed_json['t_grow'])
    data['total time [s]'].append(tot_time)
    data['% hs [s]'].append(round(100*sum(parsed_json['t_hitting_set'])/tot_time, 2))
    data['% sat [s]'].append(round(100*sum(parsed_json['t_sat_check'])/tot_time,2))
    data['% grow [s]'].append(round(100*sum(parsed_json['t_grow'])/tot_time, 2))
    data['s_hs'].append(parsed_json['s_hs']),
    data['s_grow'].append(parsed_json['s_grow'])
    data['avg_s_hs'].append(round(mean(parsed_json['s_hs']), 2)),
    data['avg_s_grow'].append(round(mean(parsed_json['s_grow']),2))
    data['OMUS'].append(parsed_json['omus'])
    data["count_clauses"].append(parsed_json['parameters']["count_clauses"] if "count_clauses" in parsed_json['parameters'] else None)
    data["best_unit_literal"].append(parsed_json['parameters']["best_unit_literal"] if "best_unit_literal" in parsed_json['parameters'] else None)
    data["best_counter_literal"].append(parsed_json['parameters']["best_counter_literal"] if "best_counter_literal" in parsed_json['parameters'] else None)
    data["sorting"].append(parsed_json['parameters']["sorting"] if "sorting" in parsed_json['parameters'] else None)
    data["extension"].append(parsed_json['parameters']["extension"] if "extension" in parsed_json['parameters'] else None)
    data["cutoff_main"].append(parsed_json['parameters']["cutoff_main"] if "cutoff_main" in parsed_json['parameters'] else None)
    data["cutoff"].append(parsed_json['parameters']["cutoff"] if "cutoff" in parsed_json['parameters'] else None)
    data["h_inc"].append(parsed_json['parameters']["h_inc"] if "h_inc" in parsed_json['parameters'] else None)
    data["max_restart"].append(parsed_json['parameters']["max_restart"] if "max_restart" in parsed_json['parameters'] else None)
    data["s_inc"].append(parsed_json['parameters']["s_inc"] if "s_inc" in parsed_json['parameters'] else None)
    data["pb_restarts"].append(parsed_json['parameters']["pb_restarts"] if "pb_restarts" in parsed_json['parameters'] else None)
    data["sp"].append(parsed_json['parameters']["sp"] if "sp" in parsed_json['parameters'] else None)


data["count_clauses"] = [ None if val == None else next(name for name, value in vars(ClauseCounting).items() if value == int(val)) for val in data["count_clauses"]]
data["best_unit_literal"]  = [ None if val == None else next(name for name, value in vars(UnitLiteral).items() if value == int(val)) for val in data["best_unit_literal"]]
data["best_counter_literal"] = [ None if val == None else next(name for name, value in vars(BestLiteral).items() if value == int(val)) for val in data["best_counter_literal"]]
data["sorting"] = [ None if val == None else next(name for name, value in vars(ClauseSorting).items() if value == int(val)) for val in data["sorting"]]
    
ignored_columns = ['solver']

df = pd.DataFrame (data, columns = [column for column in data if column not in ignored_columns])
df.sort_values(['p','type','ext','clauses','OMUS',"count_clauses","best_unit_literal",
                "best_counter_literal","sorting","extension","creation date"],
                ascending = (True, True, True, True, True, True, True, True, True, True, False), inplace = True) 
df.drop_duplicates( ['p','type','ext','clauses','OMUS',"count_clauses","best_unit_literal",
                "best_counter_literal","sorting","extension","creation date"], keep='First', inplace = True)
df = df.sort_values(["clauses", "steps", "total time [s]"], ascending = (False, True, True))
#qgrid_widget = qgrid.show_grid(df, show_toolbar=True)
#qgrid_widget

ValueError: keep must be either "first", "last" or False

In [ ]:
df_omus_only = df[(df.OMUS == True)]
#df_omus_only = df

qgrid_widget = qgrid.show_grid(df_omus_only, show_toolbar=True)
qgrid_widget

In [ ]:
df_grouped = df_omus_only.groupby('p',as_index=False)['total time [s]'].min().rename(columns={'p':'p','total time [s]' : 'min_time'})

df_merged = df_grouped.merge(df_omus_only, on='p', how='right')
df_merged = df_merged[(df_merged['total time [s]'] == df_merged['min_time'])]
best_selected_columns = ['p',
     'total time [s]', 
     'type', 
     'extension', 
     'clauses', 
     'steps', 
     "count_clauses", 
     "best_unit_literal", 
     "best_counter_literal", 
     "sorting"
    ]
df_best_only_selected_columns = df_merged[best_selected_columns]

df_best_only_selected_columns.sort_values(
    ["clauses", "p", "total time [s]"], 
    ascending = (False, True, True))
qgrid_widget = qgrid.show_grid(df_best_only_selected_columns, show_toolbar=True)
qgrid_widget

In [ ]:
#new_df = old_df[((old_df['C1'] > 0) & (old_df['C1'] < 20)) & ((old_df['C2'] > 0) & (old_df['C2'] < 20)) & ((old_df['C3'] > 0) & (old_df['C3'] < 20))]
df_selected = df[(( df['extension'] == 'greedy_param') & (df['OMUS'] == True))].sort_values(
    ["clauses", "p", "total time [s]"], 
    ascending = (False, True, True))
df_selected = df_selected[(df_selected['% hs [s]'] < 50)]
qgrid_widget = qgrid.show_grid(df_selected, show_toolbar=True)
qgrid_widget